In [1]:
#! /usr/bin/env python

import random
import numpy as np
import matplotlib.pyplot as plt
import time
from decimal import Decimal  # to deal with very large floats

from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interact
import ipywidgets as widgets

In [2]:
# define phi as f is at time t = 0 - in particular, phi defines border conditions
def phi(x, y, L):
    return (x + y)/(2*L)  # phi will always be between 0 and 1

In [3]:

def t_iterations(x, y, L, t):
    """Simulate the random walk during n iterations, starting from point (x, y).
    Return the product of the values taken by all branches during the walk from time 0 to t in a list of length t+1.
    """
    values = [phi(x, y, L)] + [0]*t
    aliveBranches = np.zeros((L+1, L+1))  # the number of alive processes at position 0 <= x,y <= L
    aliveBranches = [[x,y]]  # initially we have one branch
    for n in range(1, t+1):
        if aliveBranches == []:
            # all branches have died. The process will no longer evolve,
            # and we can keep the values initialized to 0 for all future times
            return values
        
        branchesToDelete = []  # indices of the branches that will die during this iteration
        branchesToAdd = []  # coordinates of the branches that be created (by duplication) during this iteration
        for branchId, (x, y) in enumerate(aliveBranches):
            if not (x == 0 or x == L or y == 0 or y == L):
                # we are neither on the border nor in the cemetery
                rand = random.random()
                if 0 <= rand < 1/12:
                    x -= 1  # go left
                    aliveBranches[branchId] = [x, y]
                elif 1/12 <= rand < 2/12:
                    x += 1  # go right
                    aliveBranches[branchId] = [x, y]
                elif 2/12 <= rand < 3/12:
                    y -= 1  # go down
                    aliveBranches[branchId] = [x, y]
                elif 3/12 <= rand < 4/12:
                    y += 1  # go up
                    aliveBranches[branchId] = [x, y]
                elif 1/3 <= rand < 2/3:
                    # This branch dies
                    branchesToDelete.append(branchId)
                else:
                    # Duplicate this branch
                    branchesToAdd.append([x,y])

        # Remove the dead branches
        aliveBranches = [branch for i, branch in enumerate(aliveBranches) if i not in branchesToDelete]
        # Add the newly created branches
        aliveBranches += branchesToAdd
        
        # Compute the product of the values of phi for each of our alive branches
        if aliveBranches != []:
            values[n] = 1  # initialize product
            for x, y in aliveBranches:
                values[n] *= phi(x, y, L)
    return values

In [4]:
def monte_carlo(x, y, L, K, t):    
    """Simulate K times the evolution up to time t starting from point
    (x, y), and return the average result in a list of length t."""
    temporary_sum = [0]*(t+1)
    for i in range(K):
        sample = t_iterations(x, y, L, t)  # list of length t + 1
        temporary_sum = [current + new for current, new in zip(temporary_sum, sample)]
    return [Decimal(x)/Decimal(K) for x in temporary_sum]

In [5]:
def approximate_solution(L, K, t):
    """Approximate solution at time t.
    Return a list of length t, containing the approximate solution (array of size (L, L) for all intermediate times)"""
    averages = [np.zeros((L, L)) for _ in range(t+1)]
    start = time.time()
    for x in range(L):
        for y in range(L):
            # The square associated with (x, y) is the one whose bottom left
            # corner is at (x, y)
            this_pos_monte_carlo = monte_carlo(x, y, L, K, t) # list of values until from 0 to t
            for n in range(t+1):
                averages[n][x][y] = this_pos_monte_carlo[n]
        nowm = int((time.time() - start) // 60)
        nows = int((time.time() - start) % 60)
        print("x = {:2}/{} done, elapsed time = {:2}m {:2}s".format(x, L-1, nowm, nows))
    return averages

In [6]:
def display(L, K, t):
    approx_intermediate_images = approximate_solution(L, K, t)
    def _show(frame=widgets.IntSlider(min=0,max=t-1,step=1,value=0)):
        fig, ax = plt.subplots(figsize=(7,7))
        ax.imshow(approx_intermediate_images[frame][:][:].T, origin="lower",
                   extent=[0, 1, 0, 1], cmap="jet")
    interact(_show)

In [7]:
def test():
    """Main function."""
    L = 30  # discretize with squares of length 1/L
    K = 200  # number of simulations per point to compute the average result
    t = 100
    display(L, K, t)
    
if __name__ == "__main__":
    test()

x =  0/29 done, elapsed time =  0m  1s
x =  1/29 done, elapsed time =  0m  2s
x =  2/29 done, elapsed time =  0m  2s
x =  3/29 done, elapsed time =  0m  3s
x =  4/29 done, elapsed time =  0m  4s
x =  5/29 done, elapsed time =  0m  5s
x =  6/29 done, elapsed time =  0m  6s
x =  7/29 done, elapsed time =  0m  7s
x =  8/29 done, elapsed time =  0m  7s
x =  9/29 done, elapsed time =  0m  8s
x = 10/29 done, elapsed time =  0m  9s
x = 11/29 done, elapsed time =  0m 10s
x = 12/29 done, elapsed time =  0m 11s
x = 13/29 done, elapsed time =  0m 12s
x = 14/29 done, elapsed time =  0m 13s
x = 15/29 done, elapsed time =  0m 13s
x = 16/29 done, elapsed time =  0m 14s
x = 17/29 done, elapsed time =  0m 15s
x = 18/29 done, elapsed time =  0m 16s
x = 19/29 done, elapsed time =  0m 17s
x = 20/29 done, elapsed time =  0m 18s
x = 21/29 done, elapsed time =  0m 19s
x = 22/29 done, elapsed time =  0m 19s
x = 23/29 done, elapsed time =  0m 20s
x = 24/29 done, elapsed time =  0m 21s
x = 25/29 done, elapsed t

interactive(children=(IntSlider(value=0, description='frame', max=99), Output()), _dom_classes=('widget-intera…